In [1]:
import pandas as pd
import re
import numpy as np
import regex

In [2]:
from orv_cleanup_utils import *
from add_lib import *
from agent_classifier import classify

ModuleNotFoundError: No module named 'agent_classifier'

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df = pd.read_csv('../scraping/csv/ОРВ.1_10.csv', index_col=0)
df

,project_id,foiv,7.1,7.2,7.1_other,7.2_other,7.3,7.1.1_micro_quantity,7.1.1_micro_percent,7.1.1_small_quantity,7.1.1_small_percent,7.1.1_med_quantity,7.1.1_med_percent,7.1.1_large_quantity,7.1.1_large_percent,7.1.2,7.1.3,7.1.4_main_descr_1,7.1.4_main_descr_2,7.1.4_info_descr_1,7.1.4_info_descr_2,7.1.4_benefit_descr_1,7.1.4_benefit_descr_2,7.1.5_main_once_1,7.1.5_main_regular_1,7.1.5_main_once_2,7.1.5_main_regular_2,7.1.5_info_once_1,7.1.5_info_regular_1,7.1.5_info_once_2,7.1.5_info_regular_2,7.1.5_benefit_once_1,7.1.5_benefit_regular_1,7.1.5_benefit_once_2,7.1.5_benefit_regular_2,7.1.6_total_once,7.1.6_total_regular,7.1.6_total_benefit_once,7.1.6_total_benefit_regular,7.1.7,7.1.8_np_measures,7.1.8_org_measures,7.1.9_year_1,7.1.9_micro_prognosis_year_1,7.1.9_small_prognosis_year_1,7.1.9_med_prognosis_year_1,7.1.9_year_2,7.1.9_micro_prognosis_year_2,7.1.9_small_prognosis_year_2,7.1.9_med_prognosis_year_2,9.4,9.5,9.6,9.7,9.8,9.9,14.1,14.5,14.6,14.7,14.2,14.3,14.4,2.1
0,97110,Минпромторг России,юридические лица и индивидуальные предприниматели,юридические лица и индивидуальные предпринимат...,федеральные органы исполнительной власти,"федеральные органы исполнительной власти, осущ...",источники данных: отсутствуют (место для текс...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,расходов (возможных поступлений) не предусмотрено,расходов (возможных поступлений) не предусмотрено,расходов (возможных поступлений) не предусмотрено,расходов (возможных поступлений) не предусмотрено,иные сведения о расходах (возможных поступлени...,источники данных: отсутствует (место для текст...,отсутствует,информация о программах мониторинга и иных спо...,отсутствует,описание источников информации для расчета пок...,отсутствует,отсутствует,отсутствует,средняя
1,98158,Минсельхоз России,физические и юридические лица: владельцы мелио...,неограниченнй круг лиц,-----------------,-------------------------,источники данных: правоприменительная практика...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,--------------------,-------------,--------------,--------------,иные сведения о расходах (возможных поступлени...,источники данных: ----------------- (место для...,1.создание механизма сбора и систематизации ин...,информация о программах мониторинга и иных спо...,------------,описание источников информации для расчета пок...,количество подписанных актов приемки в эксплуа...,разы (количество повторений),механический подсчет,средняя
2,95647,Минфин России,участники внешнеэкономической деятельности до...,информация отсутствует,-,-,источники данных: отсутствует (место для текс...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,иные сведения о расходах (возможных поступлени...,источники данных: - (место для текстового опис...,информация отсутствует,информация о программах мониторинга и иных спо...,-,описание источников информации для расчета пок...,-,-,-,средняя
3,100057,Минфин России,"саморегулируемая организация аудиторов, минист...","1 саморегулируемая организация аудиторов, 2 фе...",-,-,источники данных: государственный реестр самор...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,"введение новых функций, полномочий, обязанност...",0,0,0,иные сведения о расходах (возможных поступлени...,источники данных: информация отсутствует (мест...,см. п. 1.6,информация о программах мониторинга и иных спо...,0,описание источников информации для расчета пок...,0.0,0.0,0.0,средняя
4,96660,Минтранс России,владельцы инфраструктур железнодорожного транс...,не представляется возможным,данные о субъектном составе группы участников ...,информация отсутствует,источники данных: отсутствуют (место для текс...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,принятие федерального закона не приведет к изм...,информация отсутствует,информация отсутствует,информация отсутствует,иные сведения о 

# Перечислим столбцы, которые будем анализировать

In [4]:
quantity_cols_totals = [
    '7.1.6_total_once',
    '7.1.6_total_regular',
    '7.1.6_total_benefit_once',
    '7.1.6_total_benefit_regular',
    '7.2',
    '7.2_other',
    '9.5',
    '9.6',
    '9.7'
]

info_src_cols = [
    '7.1.7',
    '7.3',
    '9.9',
]

non_empty_cols = [
    '14.2',
    '14.3',
    '14.4',
]

actor_cols = [
    '7.1',
    '7.1_other',
]

goals_cols = [
    '14.1'
]

helper_cols = [
    '2.1'
]

cols_for_today = (
    ['project_id', 'foiv'] +
    quantity_cols_totals +
    info_src_cols +
    non_empty_cols +
    helper_cols +
    actor_cols
    )

In [5]:
today_df = df[cols_for_today]
for col in quantity_cols_totals:
    today_df.loc[:, col] = today_df.loc[:, col].apply(remove_junk)

/Users/alisa/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [6]:
today_df[quantity_cols_totals]

,7.1.6_total_once,7.1.6_total_regular,7.1.6_total_benefit_once,7.1.6_total_benefit_regular,7.2,7.2_other,9.5,9.6,9.7
0,,,,,юридические лица и индивидуальные предпринимат...,"федеральные органы исполнительной власти, осущ...",расходов (возможных поступлений) не предусмотрено,расходов (возможных поступлений) не предусмотрено,расходов (возможных поступлений) не предусмотрено
1,,,,,неограниченнй круг лиц,,,,
2,,,,,информация отсутствует,,,,
3,,,,,"1 саморегулируемая организация аудиторов, 2 фе...",,0,0,0
4,,,,,не представляется возможным,информация отсутствует,информация отсутствует,информация отсутствует,информация отсутствует
...,...,...,...,...,...,...,...,...,...
756,,,,,7,"1 федеральный орган исполнительной власти, 1 к...",отсутствуют,отсутствуют,отсутствуют
757,,,,,"все участники отношений, указанных в пункте 7.1","все участники отношений, указанных в пункте 7.1.",отсутствуют,в рамках текущего финансирования,не ожидаются
758,,,,,"все участники отношений, указанных в пункте 7.1","все участники отношений, указанных в пункте 7.1.",отсутствуют,в рамках текущего финансирования,не ожидаются
759,,,,,количество участников оценить не представляетс...,,нет,нет,нет


In [7]:
report_df = df[['project_id', 'foiv']].copy()

# Классификация текстовых данных

## Акторы

In [8]:
report_df.loc[:, '7.1'] = today_df['7.1'].astype(str).apply(classify)
report_df.loc[:, '7.1_other'] = today_df['7.1_other'].astype(str).apply(classify)

## Метрики (14.2-14.4)

In [9]:
no_info_list = (
        'отсутствуют',
        'отсутствует',
        'отсутсвует', 
        'отсутвует', 
        'отсуттсвуют',
        'отсутсвуют',
        'не предусмотрены',
        'не предусмотрено',
        'не предусматриваются',
        'н предусмотрены',       
        'не применимы',     
        'нет',     
        'не потребуются',
        'не требуется',
        'не требуюется',        
        'не изменятся',
        'не меняются',  
        'не предполагается',
        'не ожидается',
        'не ожидаются', 
        'не имеется',
        'не имеются',
        'нет',
        'информация отсутствует',
        'нет данных',
        'информация уточняется',
        'не установлены',
        'не определены',
        'не применимы',
        'не прослеживается',
    )

import string

def simple_clean(text):
    lines = text.splitlines()
    text = ' '.join(line.strip().lower() for line in lines if line.strip())
    return text

def remove_punct(text):
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    no_punct = text.translate(translator).strip()
    return no_punct

def normy(text):
    text = remove_punct(simple_clean(text))
    if text in no_info_list or not re.search('[а-я]+', text):
        return 'invalid'
    else:
        return 'valid'
    
for col in ['14.2', '14.3', '14.4']:
    report_df.loc[:, col] = today_df[col].astype(str).apply(normy)

# Количественные данные

In [10]:
def numbers_only(text):
    text = str(text)
    number_re = re.compile('^[\d\,\.]+$')
    if number_re.search(text):
        return True
    else:
        return False
    
def numbers(text):
    text = str(text)
    if numbers_only(text):
        return True
    number_re = re.compile('(?:\d{2}|более \d|около \d|порядка \d|млн|тыс)')
    if number_re.search(text):
        return True
    if has_digit(text):
        section_re = re.compile(
            '^(?:все лица, указанные в п. (?:7.1|6.1).? настоящего отчета|' +
            'все лица, указанные в п. 7.2.? |' +
            'все участники отношений, указанных в пункте 7.1.?)$')
        if section_re.search(text):
            return False
        impossible_re = re.compile('(возможным|невозможно)')
        if impossible_re.search(text):
            return False
        return True
    return False

def has_digit(text):
    text = str(text)
    if re.search('\d', text):
        return True
    else:
        return False

## Пункт 7

In [11]:
valid_72_pid = today_df[today_df['7.2'].apply(numbers)].project_id.to_list()

report_df.loc[:, '7.2'] = 'invalid'
valid_72_id = report_df[report_df['project_id'].isin(valid_72_pid)].index
report_df.loc[valid_72_id, '7.2'] = 'valid'

valid_72_other_pid = today_df[today_df['7.2'].apply(numbers)].project_id.to_list()

report_df.loc[:, '7.2_other'] = 'invalid'
valid_72_id = report_df[report_df['project_id'].isin(valid_72_other_pid)].index
report_df.loc[valid_72_id, '7.2_other'] = 'valid'

report_df

,project_id,foiv,7.1,7.1_other,14.2,14.3,14.4,7.2,7.2_other
0,97110,Минпромторг России,cit_no_action,foiv_no_action,invalid,invalid,invalid,invalid,invalid
1,98158,Минсельхоз России,actor_specific,no_info,valid,valid,valid,invalid,invalid
2,95647,Минфин России,foiv_cit_no_action,no_info,invalid,invalid,invalid,invalid,invalid
3,100057,Минфин России,actor_specific,no_info,invalid,invalid,invalid,valid,valid
4,96660,Минтранс России,actor_specific,mystery,invalid,invalid,invalid,invalid,invalid
...,...,...,...,...,...,...,...,...,...
756,106328,Минтранс России,actor_specific,foiv_no_action,invalid,invalid,invalid,valid,valid
757,100483,Роспотребнадзор,actor_specific,foiv_no_action,invalid,invalid,invalid,invalid,invalid
758,100483,Роспотребнадзор,actor_specific,foiv_no_action,invalid,invalid,invalid,invalid,invalid
759,101338,Минстрой России,foiv_cit_no_action,no_info,invalid,invalid,invalid,invalid,invalid


## Пункт 7.1

In [12]:
def is_valid_sum(text):
    text = str(text)
    if re.search('\d{3,}', text) or re.search('\d млн', text):
        return True
    return False

In [13]:
def validate_71(df, report_df, col):
    report_df.loc[:, col] = 'invalid'
    valid_col_pid = df[df[col].apply(is_valid_sum)].project_id.to_list()
    valid_col_id = report_df[report_df[col].isin(valid_col_pid)].index
    report_df.loc[valid_col_id, col] = 'valid'

In [14]:
for col in [
    '7.1.6_total_once',
    '7.1.6_total_regular',
    '7.1.6_total_benefit_once',
    '7.1.6_total_benefit_regular',
]:
    validate_71(today_df, report_df, col)

In [15]:
report_df

,project_id,foiv,7.1,7.1_other,14.2,14.3,14.4,7.2,7.2_other,7.1.6_total_once,7.1.6_total_regular,7.1.6_total_benefit_once,7.1.6_total_benefit_regular
0,97110,Минпромторг России,cit_no_action,foiv_no_action,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid
1,98158,Минсельхоз России,actor_specific,no_info,valid,valid,valid,invalid,invalid,invalid,invalid,invalid,invalid
2,95647,Минфин России,foiv_cit_no_action,no_info,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid
3,100057,Минфин России,actor_specific,no_info,invalid,invalid,invalid,valid,valid,invalid,invalid,invalid,invalid
4,96660,Минтранс России,actor_specific,mystery,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid
...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,106328,Минтранс России,actor_specific,foiv_no_action,invalid,invalid,invalid,valid,valid,invalid,invalid,invalid,invalid
757,100483,Роспотребнадзор,actor_specific,foiv_no_action,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid
758,100483,Роспотребнадзор,actor_specific,foiv_no_action,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid
759,101338,Минстрой России,foiv_cit_no_action,no_info,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid,invalid


## Пункт 9

In [16]:
cols =  [
    '9.5',
    '9.6',
    '9.7']

In [17]:
reason_snippet = '''
        'не потребуются в связи с тем, что похожее регулирование в отношении осетровых видов рыб осуществляется в рамках приказа минсельхоза россии от  22 апреля 2013 г. № 190',
        'в связи с принятием проекта приказа новые расходы не возникнут, так как похожее регулирование в отношении осетровых видов рыб осуществляется в рамках приказа минсельхоза россии от  22 апреля 2013 г. № 190',
        'в связи с изданием проекта приказа дополнительные поступления не возникнут, в связи с тем, что похожее регулирование в отношении осетровых видов рыб осуществляется в рамках приказа минсельхоза россии от  22 апреля 2013 г. № 190',
        'не предполагаются',  
'''

excuses_list = (
        'отсутствуют',
        'отсутствует',
        'отсутсвует', 
        'отсутвует', 
        'отсуттсвуют',
        'отсутсвуют',
        'расходы (возможные поступления) бюджетов бюджетной системы российской федерации, связанные с принятием акта, отсутствуют',
        
        'не предусмотрены',
        'не предусмотрено',
        'расходов (возможных поступлений) не предусмотрено',
        'не предусматриваются',
        'расходов (возможных поступлений) не предусматривается',
        'дополнительных расходов из федерального бюджета не потребуется',
        'н предусмотрены',
        
        'нет',
        
        'не повлечет',
        'не повлечет расходов бюджета', 
        
        'не потребуются',
        'не требуется',
        'не требуюется', 
        'принятие и реализация проекта постановления не потребуют дополнительных расходов федерального бюджета, а также не приведут к потерям доходов бюджетов бюджетной системы российской федерации',       
        'в рамках текущего финансирования, в связи с чем дополнительных расходов не потребуется', 
        'дополнительные расходы не потребуются', 
        'принятие и реализация проекта приказа не потребует дополнительных расходов федерального бюджета',
        'не потребует выделения финансовых средств из федерального бюджета',
        
        'не изменятся',
        'не меняются',
        'доходы и расходы бюджетов бюджетной системы фактически не изменятся',
        
        'не предполагается',
        
        'в соответствии с бюджетом на соответствующий календарный год',
        'в рамках текущего финансирования', 
        
        'не предвидится, все в рамках установленной штатной численности и лимитов бюджетных ассигнований, предусмотренных росрыболовству законом о федеральном бюджете.',
        
        'не ожидается',
        'не ожидаются', 
        
        'не имеется',
        'не имеются',
    )


def normalize_absence(text):
    
    text = str(text)
    
    text.replace('.', '')
    
    absence_re_str = [
        '(?:отсутствует){e<3}',
        '(?:не предусмотрено){e<3}',
        '(?:нет){d<2}',
        '(?:не повлечет){e<3}',
        '(?:не (?:по)требует)(?:ся){e<3}',
        '(?:не (?:из|по)меняют(?:ся)){e<3}',
        '(?:не предполагает(?:ся)){e<3}',
        '(?:не предвидит(?:ся)){e<3}',
        '(?:не ожидает(?:ся)){e<3}',
        '(?:не имеет(?:ся)){e<3}',
        '(?:в соответствии с){e<3}',
        '(?:в рамках){e<2}',        
    ]
    
    absent_matches = [re.search(re_str, text) for re_str in absence_re_str]
    
    reason_re_str = [
        '(?:поскольку){e<2}',
        '(?:так как){e<2}',
        '(?:в связи с тем, что){e<2}',
    ]
    
    #if any(absent_matches):
    #    reason_matches = [re.search(re_str, text) for re_str in reason_re_str]
    #    if any(reason_matches):
    #        return 'zero_for_a_reason'
    #    else:
    #        return 'zero'
        
    #else:
    #    return text

    if text in excuses_list:
        return 'zero'
    else:
        return text
    
def remove_rub(text): 
    rub_re = re.compile('( |\d)р(?:уб|ублей|убля)?(|[^а-яёй].*)?')
    text = rub_re.sub(r'\1\2', text)
    return text

def expand_zeroes(text):
    text = re.sub('тыс(яч)?', '000', text)
    text = re.sub('млн', '000000', text)
    text = re.sub('(\d)\s+(\d)', r"\1\2", text)
    return text
    
def remove_dots(text):
    text = re.sub('(\d)\.(\d)', r"\1,\2", text)
    text = text.replace('.', '')
    return text

def is_zero(text):
    return re.search('^[0\,]+$', text)

def remove_non_sums(text):
    act_number_re = re.compile('(.*)(?:№|номер) \d+(.*)')
    text = act_number_re.sub(r'\1\2', text)
    
    month_2_re = re.compile('(.*)\d\d (?:января|февраля|марта|апреля|июня|июля|августа|сентября|октября|ноября|декабря)(.*)')
    text = month_2_re.sub(r'\1\2', text)
    
    month_1_re = re.compile('(.*)\d (?:января|февраля|марта|апреля|июня|июля|августа|сентября|октября|ноября|декабря)(.*)')
    text = month_1_re.sub(r'\1\2', text)
    
    year_re = re.compile('(.*)[12]\d{3}\s*г(?:ода)?(.*)')
    text = year_re.sub(r'\1\2', text)
    
    date_re = re.compile('(.*)\d{2}\.\d{2}\.\d{4}(.*)')
    text = date_re.sub(r'\1\2', text)
    
    text = re.sub('\s+', ' ', text).strip()
    #print(text)
    
    return text  
    
def validate_sum(text):
    text = str(text)
    text = remove_non_sums(text)
    #print(text)
    text = remove_dots(text)
    #print('after remove dots', text)
    text = remove_rub(text)
    #print('after remove rub', text)
    text = expand_zeroes(text)
    
    text = text.strip()
    #print('after expand zeroes', text)
    
    if text == '0':
        return 'zero'
    
    if has_digit(text):
        if is_zero(text):
            return 'zero'
        else:
            return 'valid'
    else:
        #print('before fn', [text])
        text = normalize_absence(text)
        #print('after fn', [text])
        if text == 'zero':
            #print('wtf???')
            return text
        else:
            return 'invalid';
      
ans = 'не предполагается'
#print(normalize_absence(ans))
validate_sum(ans)

'zero'

In [18]:
for col in cols:
    report_df.loc[:, col] = today_df[col].apply(validate_sum)
    
report_df.loc[:, '2.1'] = today_df['2.1']
report_df

report_df.to_csv('report.csv')

In [19]:
answers = []

for col in cols:
    answers += today_df[col].to_list()
    
from collections import Counter
counter = Counter(answers)
for ans in counter.most_common():
    print(ans)

('', 579)
('отсутствуют', 482)
('нет', 421)
('0', 145)
('отсутствует', 101)
('не предусмотрены', 87)
('nan', 66)
('информация отсутствует', 42)
('отсутствуют.', 27)
('нет.', 24)
('нет данных', 22)
('не повлечет', 21)
('не потребуются', 20)
('0,00 р', 18)
('не предусмотрены.', 16)
('оценка не производилась', 15)
('не требуется', 12)
('не изменятся', 12)
('не предполагается', 9)
('отсутствует.', 9)
('0.00 руб', 6)
('0.00', 6)
('0,00р', 6)
('информация уточняется', 6)
('в соответствии с бюджетом на соответствующий календарный год', 6)
('рассчитать не представляется возможным', 5)
('данные отсутствуют', 4)
('не потребуются.', 4)
('не предусматриваются', 4)
('количественные данные отсутствуют', 4)
('расходов (возможных поступлений) не предусмотрено', 3)
('дополнительные поступления в бюджет от указанной административной процедуры в настоящее время оценить не представляется возможным, поскольку количество экспертиз будет зависеть от пользователей недр, представляющих соответствующие материал

# Источники информации

In [20]:
info_src_cols = [
    '7.1.7',
    '7.3',
    '9.9',
]

answers = []

for col in info_src_cols:
    answers += today_df[col].apply(remove_junk).to_list()
    
from collections import Counter
counter = Counter(answers)
for ans in counter.most_common():
    print(ans)

('', 977)
('отсутствуют', 347)
('нет', 113)
('отсутствует', 75)
('росрыболовство', 57)
('отсутствуют.', 51)
('информация отсутствует', 34)
('экспертная оценка', 14)
('данные росрыболовства', 13)
('отсутствует.', 13)
('росгидромет', 11)
('минстрой россии', 11)
('нет.', 10)
('минпромторг россии', 10)
('источники данных отсутствуют', 10)
('по данным фонда социального страхования российской федерации за 2018 г.', 9)
('федеральная служба по надзору в сфере здравоохранения', 9)
('информация о точном количестве участников отношений отсутствует', 9)
('информация росрыболовства', 9)
('минсельхоз россии', 9)
('статистические данные ростехнадзора и территориальных органов, реестр лицензий', 8)
('федеральное законодательство, данные статистики', 8)
('самостоятельные исследования', 7)
('нет данных', 7)
('действующее законодательство в области пожарной безопасности', 7)
('новых расходов не предусмотрено', 7)
('официальной статистической отчетностью не предусмотрены', 7)
('федеральный закон от 09.02.

In [21]:
def is_absent(text):
    text = remove_junk(text)
    for aw in absence_list:
        if aw in text:
            return True
    return False



In [22]:
report_df.loc[:, '7.1_text'] = today_df['7.1']
report_df.loc[:, '7.1_other_text'] = today_df['7.1_other']
report_df.to_csv('mystery.csv')